In [1]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

# Machine Learning Models
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

# Model Evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix

from sklearn.metrics import roc_auc_score, roc_curve, mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from sklearn.metrics import make_scorer

from sklearn.metrics import classification_report, accuracy_score
import random
random.seed(100)

import time
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'ODBC Driver 17 for SQL Server', 'SQL Server Native Client RDA 11.0', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


### Setting up SQL database

In [2]:
def create_sql_connection(server, database, username, password, driver='{ODBC Driver 17 for SQL Server}'):
    """
    Establish a connection to a SQL Server database using pyodbc.

    Parameters:
    - server (str): The SQL Server address (e.g., 'localhost' or server IP).
    - database (str): The name of the database you want to connect to.
    - username (str): SQL Server username.
    - password (str): SQL Server password.
    - driver (str): ODBC driver to use. Default is '{ODBC Driver 17 for SQL Server}'.s

    Returns:
    - conn: A pyodbc connection object if successful.
    """
    connection_string = f"""
        DRIVER={driver};
        SERVER={server};
        DATABASE={database};
        UID={username};
        PWD={password};
    """
    try:
        conn = pyodbc.connect(connection_string)
        print("Connection established successfully!")
        return conn
    except Exception as e:
        print(f"Failed to connect to the database. Error: {e}")
        return None

In [3]:
def query_data(conn, query):
    """
    Execute a SQL query and fetch results as a pandas DataFrame.
    
    Parameters:
    - conn: A pyodbc connection object.
    - query (str): The SQL query to be executed.
    
    Returns:
    - df: A pandas DataFrame containing the query result.
    """
    start_time = time.time()  # Start time measurement
    try:
        cursor = conn.cursor()
        cursor.execute(query)
        
        # Fetch all results from the query
        rows = cursor.fetchall()
        
        # Get column names from cursor
        columns = [desc[0] for desc in cursor.description]
        
        # Create a pandas DataFrame from the results
        df = pd.DataFrame.from_records(rows, columns=columns)
        
    except pyodbc.Error as e:
        print(f"Error executing query: {e}")
        return None
    
    finally:
        cursor.close()
    
    end_time = time.time()  # End time measurement
    execution_time = end_time - start_time  # Calculate execution time
    
    # Print the DataFrame and execution time
    print(f"Query executed in: {execution_time:.4f} seconds")
    
    return df  

In [4]:
server = 'ROHIT'     
database = 'MedicareClaim'  
username = 'rohit_kosamkar'       
password = 'September@2024' 

In [5]:
# Establish connection
conn = create_sql_connection(server, database, username, password)

Connection established successfully!


In [6]:
bene_df  = query_data(conn, "select top 10 * from beneficiarydata")
bene_df.head()

Query executed in: 0.1103 seconds


,BeneID,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt
0,BENE11001,1943-01-01,NA,1,1,0,39,230,12,12,1,2,1,2,2,1,1,1,2,1,1,36000,3204,60,70
1,BENE11002,1936-09-01,NA,2,1,0,39,280,12,12,2,2,2,2,2,2,2,2,2,2,2,0,0,30,50
2,BENE11003,1936-08-01,NA,1,1,0,52,590,12,12,1,2,2,2,2,2,2,1,2,2,2,0,0,90,40
3,BENE11004,1922-07-01,NA,1,1,0,39,270,12,12,1,1,2,2,2,2,1,1,1,1,2,0,0,1810,760
4,BENE11005,1935-09-01,NA,1,1,0,24,680,12,12,2,2,2,2,1,2,1,2,2,2,2,0,0,1790,1200


### Set Up MLflow Tracking with SQL Database

In [7]:
import mlflow
import mlflow.sklearn
from sqlalchemy import create_engine

try:
    # Set MLflow Tracking URI using SQL Server and Windows Authentication
    mlflow.set_tracking_uri("mssql+pyodbc://ROHIT/MedicareClaim?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")
 
    # Name the experiment
    mlflow.set_experiment("MedicareClaim_Fraud_Detection")
except Exception as e:
    print(f"Connection failed: {e}")

2024/10/05 16:44:57 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2024/10/05 16:44:57 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl MSSQLImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.
INFO  [alembic.runtime.migration] Context impl MSSQLImpl.
INFO  [alembic.runtime.migration] Will assume transactional DDL.


In [8]:
import mlflow
import mlflow.sklearn
from sqlalchemy import create_engine, text
import pandas as pd

# Use a raw string to handle backslashes in the server name
mlflow_tracking_uri = (
    r"mssql+pyodbc://ROHIT/MedicareClaim"
    "?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
)
 
# Set the MLflow Tracking URI
mlflow.set_tracking_uri(mlflow_tracking_uri)
 
try:
    # Create an SQLAlchemy engine
    engine = create_engine(mlflow_tracking_uri)
 
    # Test the connection by running a simple query
    with engine.connect() as connection:
        # Use the text() function to create a SQL statement
        query = text("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES")
        result = connection.execute(query)
 
        # Fetch all the results and display them
        tables = result.fetchall()
 
        # Print the list of tables
        print("Connection successful! Here are the tables in the database:")
        for table in tables:
            print(table[0])
 
    # Name the experiment (if connection is successful)
    mlflow.set_experiment("MedicareClaim_Fraud_Detection")
 
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful! Here are the tables in the database:
Beneficiarydata
experiments
runs
tags
metrics
params
alembic_version
experiment_tags
latest_metrics
registered_models
model_versions
registered_model_tags
model_version_tags
registered_model_aliases
datasets
inputs
input_tags
trace_info
trace_tags
trace_request_metadata


### Importing data

In [8]:
data = pd.read_csv(r'../../data/validation/testing_final_data.csv')
data.shape

(135181, 49)

In [9]:
del data['Unnamed: 0']
data.head()

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,DeductibleAmtPaid,ClaimPeriod,TimeInHptal,Diagnosis Count,Procedures Count,SamePhysician,OPD_Flag,DOB,DOD,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,BirthYear,Age,Alive,ChronicDisease_Count,Total_Claims_Per_Bene,Avg_Reimbursement_Per_Bene,Age_At_Claim,Multiple_Chronic_Conditions,Claim_To_Deductible_Ratio,Total_Annual_Reimbursement,Avg_Reimbursement_By_Provider,Provider_Claim_Frequency,High_Risk_Provider
0,BENE11014,CLM67387,PRV57070,9000,1068.0,7,7,10.0,2.0,No,No,1900-01-01,2009-12-01,Female,White,Yes,Utah,780,12,12,No,Yes,Yes,No,Yes,Yes,No,Yes,No,No,No,21260,2136,120,100,1938,72.0,0,5.0,2,4530.0,72.0,1,8.419083,21380,0.0,12,1
1,BENE11017,CLM31237,PRV54750,14000,1068.0,14,14,9.0,1.0,No,No,1900-01-01,2009-12-01,Female,White,No,New Jersey,270,12,12,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,No,Yes,Yes,22000,2136,1400,840,1940,70.0,0,9.0,1,14000.0,70.0,1,13.096352,23400,0.0,38,0
2,BENE11026,CLM78930,PRV53758,2000,1068.0,4,4,9.0,0.0,No,No,1900-01-01,2009-12-01,Male,White,No,Massachusetts,20,12,12,No,No,No,No,No,No,No,Yes,No,No,No,2000,1068,0,0,1938,72.0,0,1.0,1,2000.0,72.0,0,1.870907,2000,0.0,180,0
3,BENE11031,CLM56810,PRV55825,16000,1068.0,13,13,10.0,2.0,No,No,1900-01-01,2009-12-01,Female,White,No,Oregon,200,12,12,No,Yes,Yes,No,No,No,No,Yes,Yes,No,No,23650,2136,40,0,1944,65.0,0,4.0,1,16000.0,65.0,1,14.967259,23690,0.0,118,0
4,BENE11085,CLM34625,PRV52338,19000,1068.0,11,11,8.0,0.0,No,No,1900-01-01,2009-12-01,Female,White,Yes,Georgia,470,12,12,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,No,No,19000,1068,1670,520,1963,47.0,0,9.0,1,19000.0,47.0,1,17.773620,20670,0.0,1548,0


In [10]:
data.columns


Index(['BeneID', 'ClaimID', 'Provider', 'InscClaimAmtReimbursed',
       'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag', 'DOB', 'DOD', 'Gender',
       'Race', 'RenalDiseaseIndicator', 'State', 'County',
       'NoOfMonths_PartACov', 'NoOfMonths_PartBCov', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke',
       'IPAnnualReimbursementAmt', 'IPAnnualDeductibleAmt',
       'OPAnnualReimbursementAmt', 'OPAnnualDeductibleAmt', 'BirthYear', 'Age',
       'Alive', 'ChronicDisease_Count', 'Total_Claims_Per_Bene',
       'Avg_Reimbursement_Per_Bene', 'Age_At_Claim',
       'Multiple_Chronic_Conditions', 'Claim_To_Deductible_Ratio',
       

In [14]:
imp_cols = ['BeneID', 'ClaimID', 'Provider', 'InscClaimAmtReimbursed',
       'DeductibleAmtPaid', 'ClaimPeriod', 'TimeInHptal', 'Diagnosis Count',
       'Procedures Count', 'SamePhysician', 'OPD_Flag', 
        'Gender', 'Race', 'RenalDiseaseIndicator', 'NoOfMonths_PartACov', 'NoOfMonths_PartBCov',
       'ChronicCond_Alzheimer', 'ChronicCond_Heartfailure',
       'ChronicCond_KidneyDisease', 'ChronicCond_Cancer',
       'ChronicCond_ObstrPulmonary', 'ChronicCond_Depression',
       'ChronicCond_Diabetes', 'ChronicCond_IschemicHeart',
       'ChronicCond_Osteoporasis', 'ChronicCond_rheumatoidarthritis',
       'ChronicCond_stroke', 'IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt',  'Age',
       'ChronicDisease_Count', 'Total_Claims_Per_Bene',
       'Avg_Reimbursement_Per_Bene', 'Age_At_Claim',
       'Multiple_Chronic_Conditions', 'Claim_To_Deductible_Ratio',
       'Total_Annual_Reimbursement', 'Avg_Reimbursement_By_Provider',
       'Provider_Claim_Frequency', 'High_Risk_Provider']

In [15]:
# data_encoded.drop(columns={'ClaimID', 'Provider'}).corr().to_clipboard()

In [16]:
# data['PotentialFraud'].value_counts()

In [17]:
data = data[imp_cols]
data.shape

(135181, 42)

In [15]:
data.select_dtypes(include='object').columns

Index(['BeneID', 'ClaimID', 'Provider', 'SamePhysician', 'OPD_Flag', 'Gender',
       'Race', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke'],
      dtype='object')

In [35]:
### One-hot encoding
cat_cols = ['SamePhysician', 'OPD_Flag', 'Gender',
       'Race', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke']

data_encoded = pd.get_dummies(data, columns = cat_cols,drop_first=True)
data_encoded.shape

(135181, 44)

In [36]:
data_encoded.head(2)
del data_encoded['BeneID']

In [37]:
# del data_encoded['Provider_Claim_Frequency']

In [38]:
# data_encoded['PotentialFraud'].value_counts(normalize=True)

In [39]:
model_features = ['ClaimID', 'Provider','Total_Claims_Per_Bene',
    'TimeInHptal',
    'Provider_Claim_Frequency',
    'ChronicCond_stroke_Yes',
    'DeductibleAmtPaid',
    'NoOfMonths_PartBCov',
    'NoOfMonths_PartACov',
    'OPD_Flag_Yes',
    'Diagnosis Count',
    'ChronicDisease_Count', 'Age']

In [40]:
data_encoded = data_encoded[model_features]

In [63]:
data_encoded.dtypes

ClaimID                          object
Provider                         object
Total_Claims_Per_Bene             int64
TimeInHptal                       int64
Provider_Claim_Frequency          int64
ChronicCond_stroke_Yes             bool
DeductibleAmtPaid               float64
NoOfMonths_PartBCov               int64
NoOfMonths_PartACov               int64
OPD_Flag_Yes                       bool
Diagnosis Count                 float64
ChronicDisease_Count            float64
Age                             float64
Predicted_Probability_Fraud     float64
Predicted_Label                   int64
Risk_Category                  category
Fraud_Deciles                  category
dtype: object

In [41]:
model = pd.read_pickle('../../models/fraud_claim_logistic_model.pkl')

In [42]:
model

LogisticRegression(C=0.01, solver='liblinear')

In [43]:
model.predict(data_encoded.drop(['ClaimID', 'Provider'], axis=1))

array([0, 0, 1, ..., 1, 1, 0])

In [44]:
validation_predict[:10]

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 0])

In [45]:
# Predictions
validation_predict = model.predict_proba(data_encoded.drop(['ClaimID', 'Provider'], axis=1))

# Predicted probabilities for fraud (class 1)
validation_prob_fraud = validation_predict[:,1]

# Predicted labels (0 or 1)
validation_pred_labels = model.predict(data_encoded.drop(['ClaimID', 'Provider'], axis=1))

# Add these predictions back to the validation DataFrame
data_encoded['Predicted_Probability_Fraud'] = validation_prob_fraud
data_encoded['Predicted_Label'] = validation_pred_labels


In [46]:
data_encoded.head()

,ClaimID,Provider,Total_Claims_Per_Bene,TimeInHptal,Provider_Claim_Frequency,ChronicCond_stroke_Yes,DeductibleAmtPaid,NoOfMonths_PartBCov,NoOfMonths_PartACov,OPD_Flag_Yes,Diagnosis Count,ChronicDisease_Count,Age,Predicted_Probability_Fraud,Predicted_Label
0,CLM67387,PRV57070,2,7,12,False,1068.0,12,12,False,10.0,5.0,72.0,0.387235,0
1,CLM31237,PRV54750,1,14,38,True,1068.0,12,12,False,9.0,9.0,70.0,0.404688,0
2,CLM78930,PRV53758,1,4,180,False,1068.0,12,12,False,9.0,1.0,72.0,0.520080,1
3,CLM56810,PRV55825,1,13,118,False,1068.0,12,12,False,10.0,4.0,65.0,0.479240,0
4,CLM34625,PRV52338,1,11,1548,False,1068.0,12,12,False,8.0,9.0,47.0,0.985353,1


In [56]:
# Define function to categorize based on probability range
def categorize_probabilities(prob_column):
    conditions = [
        prob_column > 0.354855,    # High Risk
        (prob_column > 0.148682) & (prob_column <= 0.354855),  # Medium Risk
        prob_column <= 0.148682      # Low Risk
    ]
    
    choices = ['High', 'Medium', 'Low']
    
    # Apply the conditions to create the categories
    return pd.cut(prob_column, bins=[0, 0.148682, 0.354855, 1.0], labels=['Low', 'Medium', 'High'], right=False)

In [57]:
# Apply the function to the 'max_probability' column
data_encoded['Risk_Category'] = categorize_probabilities(data_encoded['Predicted_Probability_Fraud'])

In [60]:
data_encoded['Fraud_Deciles'] = pd.qcut(data_encoded['Predicted_Probability_Fraud'].rank(method='first', ascending=False), q=10, labels=range(1, 11))

In [61]:
data_encoded.tail()

,ClaimID,Provider,Total_Claims_Per_Bene,TimeInHptal,Provider_Claim_Frequency,ChronicCond_stroke_Yes,DeductibleAmtPaid,NoOfMonths_PartBCov,NoOfMonths_PartACov,OPD_Flag_Yes,Diagnosis Count,ChronicDisease_Count,Age,Predicted_Probability_Fraud,Predicted_Label,Risk_Category,Fraud_Deciles
135176,CLM469576,PRV55485,3,0,2823,False,0.0,12,12,True,1.0,0.0,73.0,0.997094,1,High,1
135177,CLM483842,PRV55485,3,0,2823,False,0.0,12,12,True,4.0,0.0,73.0,0.996971,1,High,1
135178,CLM554925,PRV55485,3,0,2823,False,0.0,12,12,True,2.0,0.0,73.0,0.997053,1,High,1
135179,CLM347777,PRV55889,1,0,1312,False,0.0,12,12,True,5.0,1.0,70.0,0.776926,1,High,3
135180,CLM357675,PRV52983,1,0,459,False,0.0,12,12,True,0.0,3.0,80.0,0.215190,0,Medium,6


In [62]:
data_encoded.to_csv('../../models/validation_results_logistic.csv')